In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapely as sp
import gtfs_functions as gtfs

In [6]:
# DIRECTORY = '../data/gtfs'
DIRECTORY = input('Enter location of directory (no end slash or quotes)')
while (DIRECTORY[-1] == "/") or (DIRECTORY[-1] == "\\") or (DIRECTORY[-1] == "'") or (DIRECTORY[-1] == '"'):
       DIRECTORY = input('Enter location of directory (no end slash or quotes)')
print(DIRECTORY)

IndexError: string index out of range

In [ ]:
calendar = pd.read_csv('{}/calendar.txt'.format(DIRECTORY))
# calendar_attrs = pd.read_csv('{}/calendar_attributes.txt'.format(DIRECTORY))
# directions = pd.read_csv('{}/directions.txt'.format(DIRECTORY))
feed_info = pd.read_csv('{}/feed_info.txt'.format(DIRECTORY))
#rider_cats = pd.read_csv('{}/rider_categories.txt'.format(DIRECTORY))
routes = pd.read_csv('{}/routes.txt'.format(DIRECTORY))
#runcut = pd.read_csv('{}/runcut.txt'.format(DIRECTORY))
shapes = pd.read_csv('{}/shapes.txt'.format(DIRECTORY))
stops = pd.read_csv('{}/stops.txt'.format(DIRECTORY))
#stop_attrs = pd.read_csv('{}/stop_attributes.txt'.format(DIRECTORY))
stop_times = pd.read_csv('{}/stop_times.txt'.format(DIRECTORY))
trips = pd.read_csv('{}/trips.txt'.format(DIRECTORY))

In [ ]:
date = str(feed_info.feed_start_date[0])
date = date[:4] + '-' + date[4:6] + '-' + date[6:8]
FILENAME = 'gtfs-' + date

In [ ]:
shapes.head()

In [ ]:
trips.head()

In [ ]:
routes.head()

In [ ]:
stop_times.head()

In [ ]:
calendar.head()

In [ ]:
# calendar_attrs.head()

In [ ]:
stops.head()

In [ ]:
stop_times_df  = stop_times.drop(columns=['pickup_type', 'drop_off_type',
       'timepoint', 'continuous_pickup', 'continuous_drop_off',])

In [ ]:
stops_df = stops.drop(columns=[ 'stop_code', 'zone_id', 'stop_url', 'location_type', 'parent_station', 'wheelchair_boarding',])

In [ ]:
trips_df = trips.drop(columns=['direction_id',  'bikes_allowed',  'wheelchair_accessible',])

In [ ]:
routes_df = routes.drop(columns=['agency_id',  'route_url', 'route_color',
       'route_text_color', 'route_sort_order', 'continuous_pickup', 'continuous_drop_off',])

In [ ]:
gtfs_df = stop_times_df.merge(stops_df,how='left', on='stop_id')
gtfs_df = gtfs_df.merge(trips_df, how='left', on='trip_id')
gtfs_df = gtfs_df.merge(calendar, how='left', on='service_id')
gtfs_df = gtfs_df.merge(routes_df, how='left', on='route_id')
print(gtfs_df.shape)
gtfs_df.head()

In [ ]:
size = gtfs_df.memory_usage(index=True).sum()/1000000
print('Dataframe file size is {:.2f} MB'.format(size))

In [ ]:
gtfs_df.to_excel('./notebooks/gtfs_sheets/{}.xlsx'.format(FILENAME), index=False)

In [ ]:
#arrival_time = gtfs_df.groupby('route_long_name')['arrival_time'].agg('min')
#gtfs_df['arrival_dt'] = np.where(gtfs_df['arrival_time'][:2] == '24', str(next_date) + ' ' + gtfs_df['arrival_time'],str(date) + ' ' + gtfs_df['arrival_time'] )
gtfs_df['arrival_dt'] =  pd.to_timedelta(gtfs_df['arrival_time'])

In [ ]:
date = pd.to_datetime(date)
gtfs_df['arrival_dt'] = date +gtfs_df['arrival_dt']

In [ ]:
gtfs_df.columns

In [ ]:
DAY = pd.Timedelta('1 days')
pull_in = gtfs_df.groupby(['route_long_name','service_id'])['arrival_dt'].agg(['min', 'max'])
pull_in['time_out'] = (pull_in['max'] - pull_in['min']) #/ pd.Timedelta(hours=1)
pull_in['charging_time'] = round((DAY - pull_in['time_out']) / pd.Timedelta(hours=1),2)
pull_in['time_out'] = round((pull_in['max'] - pull_in['min']) / pd.Timedelta(hours=1),2)
pull_in

In [ ]:
fig, ax = plt.hist(pull_in['charging_time'])
plt.xlabel('Charge Time')
plt.ylabel('Number of Blcoks')
plt.title('Available Charging Time per Route')
ax.set_facecolor('white')
plt.plot()

In [ ]:
schedule = gtfs_df.groupby(['route_long_name','service_id'])[['monday','tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday',]].agg('mean')
schedule

In [ ]:
with pd.ExcelWriter('./notebooks/route_summaries/route_summary_{}.xlsx'.format(FILENAME)) as writer:
       pull_in.to_excel(writer, sheet_name='Pull-In' )
       schedule.to_excel(writer, sheet_name='Schedule')